Yes, you can use the content from the GitHub repository licensed under the Apache License 2.0 to develop your own pip library, as long as you comply with the terms of the license. Here are the key points you need to follow to ensure compliance:

1 - Provide a Copy of the License: You must include a copy of the Apache License 2.0 in your project. This can be done by including a LICENSE file in the root of your project directory.

2 - State Changes: If you modify any files from the original project, you need to include a prominent notice stating that you have changed the files. This helps in distinguishing your work from the original.

3 - Preserve Notices: You must retain all the copyright, patent, trademark, and attribution notices from the source form of the original work in your derived work. This includes any notices contained in a NOTICE file if one exists.

4 - Include the NOTICE File: If the original project includes a NOTICE file, you need to include a readable copy of the attribution notices contained within it in at least one of the following places: within a NOTICE text file distributed as part of your project, within the source code or documentation, or within a display generated by your project, if applicable.

5 - Add Your Own Notices: You can add your own copyright statements and additional notices, as long as they do not contradict the terms of the Apache License 2.0.

6 - State Compliance: You must state that your use of the original project's files is in compliance with the Apache License 2.0. This can typically be done in your project's documentation or README file.

Here's a brief example of how you can include the Apache License 2.0 in your project:

1 - LICENSE File: Create a LICENSE file in the root of your project directory with the full text of the Apache License 2.0.

2 - Notice of Changes: If you modify any files, add a comment at the top of each modified file:  
\# Modified by [Your Name] on [Date]  
\# Original file available at [URL to original file]

3 - Retain Notices: Ensure that all original notices are preserved in your project.

4 - README or Documentation: Include a statement in your README file that your project includes files licensed under the Apache License 2.0 and provide a link to the license.

By following these steps, you ensure that you are respecting the terms of the Apache License 2.0 while developing your pip library using the content from the GitHub repository.

In [53]:
import random
import cv2
import numpy as np
from skimage.morphology import flood_fill

# With prompt

In [ ]:
from segment_anything import SamPredictor, sam_model_registry

In [7]:
sam = sam_model_registry["vit_b"](checkpoint="/home/mericdemirors/Downloads/sam_vit_b_01ec64.pth")
predictor = SamPredictor(sam)

In [8]:
image = cv2.imread("/home/mericdemirors/Pictures/araba/araba.jpg")

In [9]:
predictor.set_image(image)

In [ ]:
# point_coords points can be selected interactively
# point_labels (np.ndarray or None): point labels can be color
# box (np.ndarray or None): rectangle çekilerek yapılabilir

# masks, _, _ = predictor.predict(<input_prompts>)

# Without prompt

In [10]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

In [11]:
sam = sam_model_registry["vit_b"](checkpoint="/home/mericdemirors/Downloads/sam_vit_b_01ec64.pth")
mask_generator = SamAutomaticMaskGenerator(sam)

In [12]:
image = cv2.imread("/home/mericdemirors/Pictures/araba/araba.jpg")

In [13]:
masks = mask_generator.generate(image)

# Labeling image

In [46]:
mask_preds = [x["segmentation"] for x in masks]

In [54]:
labeled_image = np.zeros(image.shape[:2], dtype=np.int16)
for e,segment_mask in enumerate(mask_preds):
    labeled_image[segment_mask] = e

In [55]:
segment_pixels = np.where(labeled_image == 0)
segment_id = labeled_image.max()+1
while len(segment_pixels[0]) != 0: # while image has pixels with value 0 which means non-labeled segment
    ri, ci = segment_pixels[0][0], segment_pixels[1][0] # get a segment pixel
    
    labeled_image = flood_fill(labeled_image, (ri, ci), segment_id, connectivity=1, in_place=True) # floodfill segment
    extracted_segment = np.array(labeled_image == labeled_image[ri][ci]).astype(np.int16) # extract only segment as binary
    extracted_segment = cv2.dilate(extracted_segment, np.ones((3,3)), iterations=1) # expand segment borders by one pixel to remove edges
    np.putmask(labeled_image, extracted_segment != 0, segment_id) # overwrite expanded segment to labeled_image

    segment_id = segment_id + 1
    segment_pixels = np.where(labeled_image == 0)

In [56]:
cv2.imshow("i", labeled_image.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()